#**Дятлов Михаил Александрович M4138c**

## Теория информации.


### Здание №1: **Преобразование Барроуза — Уилера**

### Введение.

* Все разделено на блоки. Каждый блок либо текст либо программа

* Блоки программы можно выполнять отдельно. При этом должны быть выполнены блоки, от которых зависит выполняемый блок, например импорт библиотек, которые используются в блоке или части, где инициализируются переменные, считываются файлы и т.д.

* Для выполнения блока есть кнопка слева (при наведении на блок). Так же можно нажать на блок (выделить) и нажать "ctrl" + "enter".

* В меню (сверху) есть вкладка "Среда выполнения". В ней есть возможность выбрать "Выполнить все" для запуска всей программы.

### Описание.

* Над каждым блоком кода есть блок текста, который описывает назначение кода.

* Обязательно нужно выполнить импорт.

* Часть "сжатие" можно выполнять отдельно от "извлечение".
Обе части разделены на: загрузку файлов, основной код и выгрузка получившихся файлов.

* Часть "сжатие" создает файлы с названием исходного + '.nbw', которые можно загрузить для извлечения.

* Загружать можно несколько файлов, алгоритм будет применен к каждому по очереди.

* К сожалению для больших файлов алгоритм может занять значительное время. На реализацию более оптимального ршения требуется больше вермени.

* 1ч. 30 мин. на файл book1 размером 700Кб
* 38 мин. на файл pic размером 513Кб
* 26с на файл geo размером 102Кб





#### **Импорт** 

In [ ]:
from google.colab import files
import numpy as np

### **Энтропия**


#### Загрузка файлов

In [ ]:
entropy_files = files.upload()

Saving book1 to book1
Saving book2 to book2
Saving pic to pic (1)
Saving news to news (1)
Saving obj2 to obj2 (3)
Saving bib to bib (2)
Saving geo to geo (1)
Saving trans to trans (3)
Saving paper2 to paper2 (2)
Saving progl to progl (2)
Saving paper1 to paper1 (3)
Saving progp to progp (2)
Saving progc to progc (2)
Saving obj1 to obj1 (5)


#### Подсчет энтропии

Функция энтропии

In [ ]:
def H_X(X, X_inds=None, depth=1):
  new_X = X
  if X_inds is not None:
    new_X = new_X[X_inds]
  unique, counts = np.unique(new_X, return_counts=True)
  p_x = counts/new_X.size
  if depth > 1:

    def H_x(x):
      if X_inds is not None:
        inds = np.intersect1d(X_inds, np.where(X == x)[0])+1
      else:
        inds = np.where(X == x)[0]+1
      if inds.size and inds[-1] == X.size:
        inds = inds[:-1]
      return H_X(X, inds, depth=depth-1) if inds.size > 1 else 0

    arr = np.array([], dtype=np.uint8)
    for x in unique:
      arr = np.append(arr, H_x(x))
    return (p_x * arr).sum()

  else:
    p_x = np.where(p_x == 0, 1, p_x)
    return (-p_x * np.log2(p_x)).sum()



#### Вывод таблицы

In [ ]:
print('%10s\t%4s\t%4s\t%4s' % ('', 'H(X)', 'H(X|X)', 'H(X|XX)'))
for f_key in entropy_files.keys():
  X = np.frombuffer(entropy_files[f_key], dtype=np.uint8)
  print('%10s\t%2.2f\t%2.2f\t%2.2f' % (f_key, H_X(X), H_X(X, depth=2), H_X(X, depth=3)))


          	H(X)	H(X|X)	H(X|XX)
     book1	4.53	3.58	2.81
     book2	4.79	3.75	2.74
       pic	1.21	0.82	0.71
      news	5.19	4.09	2.92
      obj2	6.26	3.87	2.27
       bib	5.20	3.36	2.31
       geo	5.65	4.26	3.46
     trans	5.53	3.35	1.93
    paper2	4.60	3.52	2.51
     progl	4.77	3.21	2.04
    paper1	4.98	3.65	2.33
     progp	4.87	3.19	1.76
     progc	5.20	3.60	2.13
      obj1	5.95	3.46	1.40


### **Сжатие**


#### Загрузка файла

In [ ]:
uploaded = files.upload()

Saving paper2 to paper2


#### Кодирование загруженных файлов


In [ ]:
def partition(arr, inds, l, r):
  v = inds[(l + r) // 2]
  v = arr[v:] + arr[:v]
  i = l
  j = r
  while i <= j:
    while True:
      a = arr[inds[i]:] + arr[:inds[i]]
      if a < v:
        del a
        i += 1
      else:
        del a
        break
    while True:
      a = arr[inds[j]:] + arr[:inds[j]]
      if a > v:
        del a
        j -= 1
      else:
        del a
        break
    if i >= j:
      break
    inds[i], inds[j] = inds[j], inds[i]
    i += 1
    j -= 1
  del v
  return j

def q_sort(arr, inds, l, r):
  if l<r:
    q = partition(arr, inds, l, r)
    q_sort(arr, inds, l, q)
    q_sort(arr, inds, q + 1, r)

def mon(i):
  if i == 0:
    return '0'
  else:
    n_log = int(np.floor(np.log2(i)))
  return '1'*n_log + '0' + bin(i - 2**n_log)[2:]

for f_key in uploaded.keys():
  with open(f_key + '.nbw', 'wb') as output_file:
    raw = uploaded[f_key]
    inds = np.arange(len(raw))
    q_sort(raw, inds, inds[0], inds[-1])
    res = np.array(bytearray(raw), dtype=np.uint8)[inds-1]
    M = list(range(256))
    for i in range(res.size):
      ind = M.index(res[i])
      res[i] = ind
      M.insert(0, M.pop(ind))

    freq = np.asarray(np.unique(res, return_counts=True)).T
    P = freq[:, 1]
    P = (P/P.sum())
    C = np.full((P.size, P.size), np.nan)
    T = np.full((P.size, P.size), np.nan)
    T[:, 0] = np.arange(P.size)
    L = np.zeros(P.size, dtype=np.uint8)
    while True:
      if np.any(P == 1):
        break
      [ind2, ind1] = sorted(np.argsort(P)[:2])
      P[ind1] = P[ind1] + P[ind2]
      P[ind2] = np.nan
      inds1 = T[ind1, np.where(np.invert(np.isnan(T[ind1,:])))[0]].astype(np.uint8)
      inds2 = T[ind2, np.where(np.invert(np.isnan(T[ind2,:])))[0]].astype(np.uint8)
      C[inds1, L[inds1]] = 0
      C[inds2, L[inds2]] = 1
      T[ind1, np.where(np.isnan(T[ind1,:]))[0][:(1-np.isnan(T[ind2,:])).sum()]] = T[ind2, np.invert(np.isnan(T[ind2,:]))]
      T[ind2, :] = np.nan
      L[T[ind1, np.where(np.invert(np.isnan(T[ind1,:])))[0]].astype(np.uint8)] += 1
    
    abc = []
    for row in C:
      abc.append(''.join(row[::-1][np.where(np.invert(np.isnan(row[::-1])))].astype(np.uint8).astype(str)))
    abc = np.array(abc)
    b_c = np.arange(256).astype(str)
    b_c[freq[:, 0]] = abc
    res = b_c[res]
    res_bits = ''.join(res)
    res_bits += '0'*(8 - len(res_bits)%8)
    res = np.array(list(map(lambda a: int(a, 2), [res_bits[i:i+8] for i in range(0, len(res_bits), 8)])), dtype=np.uint8)

    abc = '$' + np.char.array(abc) + '$'

    output_file.write(
      bytearray(str(np.where(inds==0)[0][0]), 'ascii')
      + bytearray('\n', 'ascii')
      + bytearray(''.join(np.hstack((abc.reshape(-1, 1), freq[:, 0].reshape(-1, 1))).reshape((-1))), 'ascii')
      + bytearray('\n', 'ascii')
      + res.tobytes()
    )

#### Скачать закодированные файлы

In [ ]:
for f_key in uploaded.keys():
  files.download(f_key + '.nbw')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **Извлечение**

#### Загрузка файла

In [ ]:
coded_files = files.upload()

Saving paper2.nbw to paper2 (1).nbw


#### Декодирование загруженных файлов

In [ ]:
def mon(i):
  if i == 0:
    return '0'
  else:
    n_log = int(np.floor(np.log2(i)))
  return '1'*n_log + '0' + bin(i - 2**n_log)[2:]

for f_key in coded_files.keys():
  with open(f_key.replace('.nbw', ''), 'wb') as output_file:
    coded = bytearray(coded_files[f_key])
    ind_index = coded.index(b'\n')
    ind = int(coded[:ind_index])
    coded = coded[ind_index + 1:]
    abc_index = coded.index(b'\n')
    abc = coded[:abc_index].decode('ascii').split('$')[1:]
    abc = np.array(abc).reshape((-1, 2))
    coded = coded[abc_index + 1:]
    row = ''.join(map(lambda a: '0'*(8-len(bin(a)[2:])) + bin(a)[2:], coded))

    data = []
    buf = ''
    for i in row:
      buf += i
      if np.any(abc[:, 0] == buf):
        data.append(int(abc[np.where(abc[:, 0] == buf)[0][0], 1]))
        buf = ''
    # print(data)
    M = list(range(256))
    res = []
    for i in range(len(data)):
      res.append(M[data[i]])
      M.insert(0, M.pop(data[i]))
    
    res = np.array(res, dtype=np.uint8)
    # print(res)
    freq = np.bincount(np.array(res))
    freq = freq.cumsum()-freq
    t = np.empty(len(res), dtype=np.uint64)
    for i in range(len(res)):
      t[freq[res[i]]] = i
      freq[res[i]] += 1
    j = t[ind]
    answer = np.empty(len(res), dtype=np.uint8)
    for i in range(len(res)):
      answer[i] = res[j]
      j = t[j]
    output_file.write(answer.tobytes())


#### Скачать декодированные файлы

In [ ]:
for f_key in coded_files.keys():
  files.download(f_key.replace('.nbw', ''))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>